# Preprocess VTaC dataset

In [1]:
 import pandas as pd
 import wfdb
 import matplotlib.pyplot as plt
 import numpy as np
 import h5py

Each waveform recording in the [VTaC](https://physionet.org/content/vtac/1.0/)
 dataset contains at least two electrocardiogram (ECG) leads and one or more pulsatile waveforms, such as photoplethysmogram (PPG or PLETH) and arterial blood pressure (ABP) waveforms.

In this demostration, we utilize the ECG signal from lead II and the PPG signal from PLETH waveform. The training, validation, and test sets were partitioned according to the official dataset split.


In [2]:
root_dir = "/mnt/Data2/data/vtac-a-benchmark-dataset-of-ventricular-tachycardia-alarms-from-icu-monitors-1.0/"
data_df = pd.read_csv(root_dir + '/benchmark_data_split.csv')
label_df = pd.read_csv(root_dir + '/event_labels.csv')

In [3]:
# function to get all the segments
import wfdb
import numpy as np
import neurokit2 as nk
import heartpy as hp
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew

def flat_sqi(signal, window=60, e=1e-5):
    for i in range(len(signal) - window):
        seg = signal[i:i + window]
        d = np.diff(seg)
        counter = sum([abs(a) < e for a in d])
        if counter == window - 1:
            return 1  # bad signal
    return 0  # good signal

def skewness_sqi(sig, sampling_rate, window=2, step=1, return_s=False):
    step_size = step * sampling_rate
    range_sample = window * sampling_rate
    start = 0
    end = 0
    s = []
    while start < len(sig) - range_sample:
        end = start + range_sample
        seg = sig[start:end]
        start = start + step_size
        s.append(skew(seg))
    counter = [a > 0 for a in s]  # threshold for skewness is 0, i.e. positive skewed
    if sum(counter) > 0.5 * len(s):  # if 50% skewness are positive
        if return_s:
            return 0, s  # good signal
        else:
            return 0
    else:
        if return_s:
            return 1, s
        else:
            return 1  # bad signal
        
def median_absolute_deviation(x):
    return np.median(np.abs(x - np.median(x)))

def hampel(ts, window_size=5, n=3, imputation=False):
    if type(ts) != pd.Series:
        ts = pd.Series(ts)

    ts_cleaned = ts.copy()
    k = 1.4826

    rolling_ts = ts_cleaned.rolling(window_size * 2, center=True)
    rolling_median = rolling_ts.median().bfill().ffill()
    rolling_sigma = k * (rolling_ts.apply(median_absolute_deviation).bfill().ffill())

    outlier_indices = list(np.array(np.where(np.abs(ts_cleaned - rolling_median) >= (n * rolling_sigma))).flatten())

    if imputation:
        ts_cleaned[outlier_indices] = rolling_median[outlier_indices]
        return outlier_indices, np.array(ts_cleaned)

    return outlier_indices

def ppg_quality(ppg, fs):
    if flat_sqi(ppg) == 1:
        return 0, ppg

    ppg = (ppg - ppg.mean()) / ppg.std()
    ppg_p = nk.ppg_clean(ppg, sampling_rate=fs, method='elgendi')  # third order bandpass butterworth filter

    try:
        wd, m = hp.process(ppg_p, fs, windowsize=1, reject_segmentwise=True)
    except hp.exceptions.BadSignalWarning:
        return 0, ppg

    if sum(wd['binary_peaklist']) < 5:
        return 0, ppg

    if skewness_sqi(ppg_p, fs) == 0:
        return 1, ppg_p
    else:
        return 0, ppg

def get_ecg_ppg_signals(seg_path, signal_duration=10, fs=250):
    seg_path = Path(seg_path)
    record = wfdb.rdheader(str(seg_path))
    if 'II' not in record.sig_name or not any(
            ppg in record.sig_name for ppg in ['PLETH', 'PLETH L', 'PLETH R']):
        return None

    # Skip the first 10 seconds
    initial_offset = (60*5-10) * fs
    
    segs = wfdb.rdsamp(str(seg_path), sampfrom=initial_offset, sampto=initial_offset+10*fs)
    idx_ecg = segs[1]['sig_name'].index('II')
    idx_ppg = segs[1]['sig_name'].index('PLETH')
    signal = segs[0][:, [idx_ecg, idx_ppg]]  # Get the signal segment
    ecg_mean = np.nanmean(signal[:, 0])
    ppg_mean = np.nanmean(signal[:, 1])
    
    signal[:, 0] = np.where(np.isnan(signal[:, 0]), ecg_mean, signal[:, 0])
    signal[:, 1] = np.where(np.isnan(signal[:, 1]), ppg_mean, signal[:, 1])
    
    if np.isnan(signal).any():
        return None
    
    ecg_cleaned = nk.ecg_clean(signal[:, 0], sampling_rate=fs)
    ppg_sqi, ppg_cleaned = ppg_quality(signal[:, 1], fs)
  
    return np.stack((ecg_cleaned, ppg_cleaned), axis=0)



In [5]:
train_signals = []
train_labels = []

val_signals = []
val_labels = []

test_signals = []
test_labels = []

from tqdm import tqdm 

for i in tqdm(range(len(data_df))):
    if data_df.split[i] == 'train':
        data_path = f'{root_dir}/waveforms/{data_df.record[i]}/{data_df.event[i]}'
        sample = get_ecg_ppg_signals(data_path)
        label = label_df[label_df.event == data_df.event[i]].decision
        if sample is not None:
            train_signals.append(sample)
            train_labels.append(label)
    elif data_df.split[i] == 'val':
        data_path = f'{root_dir}/waveforms/{data_df.record[i]}/{data_df.event[i]}'
        sample = get_ecg_ppg_signals(data_path)
        label = label_df[label_df.event == data_df.event[i]].decision
        if sample is not None:
            val_signals.append(sample)
            val_labels.append(label)
    else:
        data_path = f'{root_dir}/waveforms/{data_df.record[i]}/{data_df.event[i]}'
        sample = get_ecg_ppg_signals(data_path)
        label = label_df[label_df.event == data_df.event[i]].decision
        if sample is not None:
            test_signals.append(sample)
            test_labels.append(label)

train_labels_new = np.array(train_labels)
train_labels_new = [int(value) for value in train_labels_new]

val_labels_new = np.array(val_labels)
val_labels_new = [int(value) for value in val_labels_new]

test_labels_new = np.array(test_labels)
test_labels_new = [int(value) for value in test_labels_new]


100%|██████████| 5037/5037 [02:38<00:00, 31.83it/s]


In [6]:
import h5py
from tqdm import tqdm
import numpy as np

import h5py
# Create HDF5 file and save data

output_h5_path = 'vtac_train.h5'
with h5py.File(output_h5_path, 'w') as h5f:
    h5f.create_dataset('tracings', data=train_signals)
    h5f.create_dataset('labels', data=np.array(train_labels_new).reshape(-1,1))

output_h5_path = 'vtac_val.h5'
with h5py.File(output_h5_path, 'w') as h5f:
    h5f.create_dataset('tracings', data=val_signals)
    h5f.create_dataset('labels', data=np.array(val_labels_new).reshape(-1,1))

output_h5_path = 'vtac_test.h5'
with h5py.File(output_h5_path, 'w') as h5f:
    h5f.create_dataset('tracings', data=test_signals)
    h5f.create_dataset('labels', data=np.array(test_labels_new).reshape(-1,1))